In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mashqa-dataset/test_webmd_squad_v2_nonconsec.json
/kaggle/input/mashqa-dataset/val_webmd_squad_v2_full.json
/kaggle/input/mashqa-dataset/train_webmd_squad_v2_full.json
/kaggle/input/mashqa-dataset/test_webmd_squad_v2_consec.json
/kaggle/input/mashqa-dataset/test_webmd_squad_v2_full.json
/kaggle/input/mashqa-dataset/train_webmd_squad_v2_consec.json
/kaggle/input/mashqa-dataset/val_webmd_squad_v2_nonconsec.json
/kaggle/input/mashqa-dataset/val_webmd_squad_v2_consec.json
/kaggle/input/mashqa-dataset/train_webmd_squad_v2_nonconsec.json


In [2]:
df = pd.read_json('/kaggle/input/mashqa-dataset/train_webmd_squad_v2_full.json')

def extract_qas(entry):
    rows = []
    for para in entry['paragraphs']:
        context = para['context']
        for qa in para['qas']:
            for ans in qa['answers']:
                rows.append({
                    'question': qa['question'],
                    'context': context,
                    'answer': ans['text'],
                    'answer_start': ans.get('answer_start'),
                    'is_impossible': qa['is_impossible'],
                    'id': qa['id']
                })
    return rows

# Apply to all rows in the DataFrame
all_qas = []
for i in range(len(df)):
    all_qas.extend(extract_qas(df.iloc[i].values[1]))

qa_df = pd.DataFrame(all_qas)
qa_df.head()


,question,context,answer,answer_start,is_impossible,id
0,What are the symptoms of gastritis?,var s_context; s_context= s_context || {}; s_c...,"However, the most common symptoms include: Nau...",1688.0,False,46a31525fae6a6b203c9be1e0eb7017d
1,What does the treatment for gastritis involve?,var s_context; s_context= s_context || {}; s_c...,Treatment for gastritis usually involves: Taki...,2888.0,False,75f972ba8b72bcb5b2bd901bd7006767
2,How does an upper endoscopy help diagnose gast...,var s_context; s_context= s_context || {}; s_c...,"However, the most common symptoms include: Nau...",1688.0,False,9871bb970b4c59aba85aecad486798f1
3,How is a fecal occult blood test (stool test) ...,var s_context; s_context= s_context || {}; s_c...,This test checks for the presence of blood in ...,2800.0,False,de443e4a4712a0b6e2cc555cdb51afdc
4,What is gastritis?,var s_context; s_context= s_context || {}; s_c...,"!s_sensitive, chron ID: $('article embeded_mod...",485.0,False,a141a0d225d94a318f264aac7b6c6832


In [3]:
qa_df.iloc[0].values

array(['What are the symptoms of gastritis?',
       "var s_context; s_context= s_context || {}; s_context['wb.modimp'] = 'vidfloat'; if(webmd.useragent && webmd.useragent.ua.type === 'desktop'){ webmd.ads2.disable Initial Load(); webmd.ads2.disable Ads Init = true; $(function() { webmd.p.pim.increment(); $('.responsive-video-container').insert After('.module-social-share-container'); require(['video2/1/responsive-player/video-loader'], function(video Loader) { video Loader.init({ autoplay: webmd.useragent.ua.type === 'desktop' && ! !s_sensitive, chron ID: $('article embeded_module[type=video][align=top]:eq(0)').attr('chronic_id'), continuous Play: true, cp Options: { flyout: true }, display Ads: true, mode: 'in-article', sticky: true }) }); }); } else { $(function(){ $('.responsive-video-container').remove(); }); } Gastritis is an inflammation, irritation, or erosion of the lining of the stomach. It can occur suddenly (acute) or gradually (chronic). Gastritis can be caused by irritati

In [4]:
# Naive fix: remove content before "Gastritis is..."
def clean_context(text):
    start = text.find("Gastritis is")
    return text[start:] if start != -1 else text

qa_df['clean_context'] = qa_df['context'].apply(clean_context)


In [5]:
qa_df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,question,context,answer,answer_start,is_impossible,id,clean_context
0,What are the symptoms of gastritis?,var s_context; s_context= s_context || {}; s_c...,"However, the most common symptoms include: Nau...",1688.0,False,46a31525fae6a6b203c9be1e0eb7017d,"Gastritis is an inflammation, irritation, or e..."
1,What does the treatment for gastritis involve?,var s_context; s_context= s_context || {}; s_c...,Treatment for gastritis usually involves: Taki...,2888.0,False,75f972ba8b72bcb5b2bd901bd7006767,"Gastritis is an inflammation, irritation, or e..."
2,How does an upper endoscopy help diagnose gast...,var s_context; s_context= s_context || {}; s_c...,"However, the most common symptoms include: Nau...",1688.0,False,9871bb970b4c59aba85aecad486798f1,"Gastritis is an inflammation, irritation, or e..."
3,How is a fecal occult blood test (stool test) ...,var s_context; s_context= s_context || {}; s_c...,This test checks for the presence of blood in ...,2800.0,False,de443e4a4712a0b6e2cc555cdb51afdc,"Gastritis is an inflammation, irritation, or e..."
4,What is gastritis?,var s_context; s_context= s_context || {}; s_c...,"!s_sensitive, chron ID: $('article embeded_mod...",485.0,False,a141a0d225d94a318f264aac7b6c6832,"Gastritis is an inflammation, irritation, or e..."
...,...,...,...,...,...,...,...
27723,How do some people clean their ears?,"Some people swear by their cotton swabs, and o...",Maybe you're one of those that says you should...,NaN,False,e96a539e187b6710ca55bc8c0edf7032,"Some people swear by their cotton swabs, and o..."
27724,What products should never be used to clean ears?,"Some people swear by their cotton swabs, and o...",Swabbing or sticking pointy objects inside you...,NaN,False,32ae6d10bd578c0c7d151858b94082e8,"Some people swear by their cotton swabs, and o..."
27725,Why do our ears make wax?,"Some people swear by their cotton swabs, and o...",The only reason you should clean them is to so...,NaN,False,ca337ee19f02c1c26aeb1cf204f07411,"Some people swear by their cotton swabs, and o..."
27726,What do we know about the African plum tree fo...,"After non-melanoma skin cancer, prostate cance...","In Europe, extract from the African plum tree ...",4519.0,False,faff993be1e5ba0f459e08f521cb234c,"After non-melanoma skin cancer, prostate cance..."


In [6]:
row = qa_df.iloc[0]
start = int(row['answer_start'])
predicted_answer = row['context'][start:start+len(row['answer'])]
print(predicted_answer == row['answer'])  # Should be True


True


In [7]:
# Clean QA dataset: Question & Answer only
clean_qa_df = qa_df[['question', 'answer']].copy()

# Drop rows with missing or empty answers (just in case)
clean_qa_df = clean_qa_df.dropna(subset=['question', 'answer'])
clean_qa_df = clean_qa_df[clean_qa_df['answer'].str.strip() != '']

# Reset index
clean_qa_df = clean_qa_df.reset_index(drop=True)

# Preview
clean_qa_df.head()


,question,answer
0,What are the symptoms of gastritis?,"However, the most common symptoms include: Nau..."
1,What does the treatment for gastritis involve?,Treatment for gastritis usually involves: Taki...
2,How does an upper endoscopy help diagnose gast...,"However, the most common symptoms include: Nau..."
3,How is a fecal occult blood test (stool test) ...,This test checks for the presence of blood in ...
4,What is gastritis?,"!s_sensitive, chron ID: $('article embeded_mod..."


In [8]:
for idx, row in qa_df.iloc[:100].iterrows():
    question = row['question']
    answer = row['answer']
    
    print(f"--- QA Pair {idx + 1} ---")
    print(f"Q: {question}")
    print(f"A: {answer}\n")


--- QA Pair 1 ---
Q: What are the symptoms of gastritis?
A: However, the most common symptoms include: Nausea or recurrent upset stomach Abdominal bloating Abdominal pain Vomiting Indigestion Burning or gnawing feeling in the stomach between meals or at night Hiccups Loss of appetite Vomiting blood or coffee ground-like material Black, tarry stools To diagnose gastritis, your doctor will review your personal and family medical history, perform a thorough physical evaluation, and may recommend any of the following tests: Upper endoscopy.

--- QA Pair 2 ---
Q: What does the treatment for gastritis involve?
A: Treatment for gastritis usually involves: Taking antacids and other drugs (such as proton pump inhibitors or H-2 blockers) to reduce stomach acid Avoiding hot and spicy foods For gastritis caused by H. pylori infection, your doctor will prescribe a regimen of several antibiotics plus an acid blocking drug (used for heartburn) If the gastritis is caused by pernicious anemia, B12 vita

In [9]:
clean_qa_df.shape

(27728, 2)

In [10]:
clean_qa_df.to_csv('mash_qa_data.csv')

In [11]:
import os
import pandas as pd
from openai import OpenAI
import time
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt
from kaggle_secrets import UserSecretsClient
from openai import OpenAI


user_secrets = UserSecretsClient()

openai_key = user_secrets.get_secret("openai")

# Initialize the OpenAI client
try:
    client = OpenAI(api_key=openai_key)
except KeyError:
    print("Error: OPENAI_API_KEY environment variable not set.")
    exit()

# --- Function 1: Load Data ---
def load_data(file_path):
    """
    Loads QA pairs from a specified file.
    Assumes the file is a CSV with 'question' and 'answer' columns.
    """
    try:
        df = pd.read_csv(file_path)
        return df[['question', 'answer']].iloc[:1000].to_dict('records')
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return []

# --- Function 2: Construct Prompt Messages ---
def construct_prompt_messages(qa_pair):
    """
    Constructs the list of messages for a single API call,
    using the robust JSON prompt format.
    """
    question = qa_pair['question']
    answer = qa_pair['answer']
    
    user_content = f"""
Core Constraints:
1.  **Grounded and Natural:** All correct options MUST be directly derived from the provided 'Answer' text. You must NOT use any external knowledge. Slightly reword the information to sound natural and well-formed, rather than simply copying from the source.
2.  **Comprehensive Extraction:** Generate all possible correct options from the 'Answer' text that are suitable for a multiple-choice question. Do not leave out any potentially valid options.
3.  **Quantity Control:** If there is only a single suitable fact, create only ONE correct option. If there are multiple suitable facts, create as many correct options as possible, up to a maximum of 5.
4.  **Challenging Distractors:** All incorrect options (distractors) must be either a direct contradiction of a correct fact from the text or a statement from a completely unrelated topic.
5.  **Correct options must be well-formed, natural-sounding sentences or phrases. Do not simply copy-paste or literally rephrase the source text. Synthesize the facts into a concise and fluent statement.** All options must be seperated by ";"
6.  **Question Rephrasing:** You are permitted to make minor, stylistic changes to the original 'Question' to improve its clarity and alignment with the generated options. The core meaning and intent of the question must be preserved.
7.  **Question Phrasing:** The final `question` field should not contain phrases such as "Based on the text," "According to the provided information," or any similar wording.
8.  **Option Clarity:** In all correct and incorrect options, use the proper name of the subject (e.g., "Gastritis," "Cerebellum") instead of pronouns (e.g., "it", "these", "them" etc).

Examples:

Input QA Pair (Multi-Correct):
Question: What is a bone marrow transplant?
Answer: A bone marrow transplant is a procedure that infuses healthy blood stem cells into your body. This procedure is also known as a stem cell transplant or a hematopoietic stem cell transplant. This can be used to treat certain types of cancer and other diseases.

Output JSON:
{{
  "question": "Which of the following statements about a bone marrow transplant are correct? (Select all that apply)",
  "correct_options": [
    "It involves the infusion of healthy blood stem cells.";
    "It can be used to treat certain cancers and other diseases.";
    "The procedure may also be referred to as a stem cell transplant.";
  ],
  "distractor_options": [
    "It is a procedure that removes stem cells from the patient's body.";
    "It is a surgical procedure to repair a damaged bone.";
    "It is a type of organ donation.";
  ]
}}

Input QA Pair (Single-Correct):
Question: What is the primary function of the cerebellum?
Answer: The cerebellum, or "little brain," is a region of the brain that plays an important role in motor control. It does not initiate movement, but it contributes to coordination, precision, and accurate timing.

Output JSON:
{{
  "question": "What is a key function of the cerebellum?",
  "correct_options": [
    "It aids in motor coordination and the timing of movements."
  ],
  "distractor_options": [
    "It is responsible for initiating all voluntary movements.";
    "It controls the body's autonomic nervous system.";
    "It is the main center for sensory perception.";
  ]
}}

Now, your turn. Generate the JSON output for the following QA pair.

Input QA Pair:
Question: {question}
Answer: {answer}
"""
    messages = [
        {
            "role": "system",
            "content": "You are an expert at creating difficult multiple-choice questions with multiple correct answers and challenging distractors. Your task is to generate a question based on a provided medical QA pair. You MUST output your response as a single, valid JSON object. Do NOT include any additional text or markdown outside of the JSON."
        },
        {
            "role": "user",
            "content": user_content
        }
    ]
    return messages

# --- Function 3: Call GPT-4o-mini with Robustness ---
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def call_gpt4o_mini(messages):
    """
    Sends a single prompt to the GPT-4o-mini API with robustness for errors and rate limits.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            response_format={"type": "json_object"},
            temperature=0.7,
            n=1
        )
        # Ensure the response content is a JSON string before returning
        content = response.choices[0].message.content
        return json.loads(content)
    except Exception as e:
        print(f"An error occurred during the API call: {e}")
        # Re-raise the exception to trigger the retry logic
        raise e

# --- Function 4: Orchestrate the Process ---
def process_dataset(input_file, output_file, batch_size=50):
    """
    Main function to load the dataset, process it in batches, and save the results.
    """
    qa_pairs = load_data(input_file)
    if not qa_pairs:
        return

    processed_results = []
    num_rows = len(qa_pairs)
    
    print(f"Starting to process {num_rows} rows in batches of {batch_size}.")

    for i in range(0, num_rows, batch_size):
        batch = qa_pairs[i:i + batch_size]
        print(f"Processing batch starting at row {i+1}...")

        # Process each QA pair in the batch individually
        for qa_pair in batch:
            messages_for_api = construct_prompt_messages(qa_pair)
            try:
                # Call the robust API function for each prompt
                generated_json = call_gpt4o_mini(messages_for_api)

                generated_json['clean_context'] = qa_pair['answer']

                
                processed_results.append(generated_json)
                print(f"  Processed QA pair {i+1 + batch.index(qa_pair)} successfully.")
            except Exception as e:
                print(f"  Failed to process QA pair {i+1 + batch.index(qa_pair)} after multiple retries. Skipping.")
        
        # A small delay between batches is good practice
        time.sleep(0.01)

    # Save all results to a single JSON file
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(processed_results, f, indent=4)
    
    print(f"Processing complete. {len(processed_results)} results saved to {output_file}")

# --- Main Execution Block ---
if __name__ == "__main__":
    input_csv_file = '/kaggle/working/mash_qa_data.csv'
    output_json_file = 'generated_questions.json'
    
    batch_size = 50 
    input_df = pd.read_csv(input_csv_file)
    print(f"Dataset Rows: {input_df.shape[0]}")
    process_dataset(input_csv_file, output_json_file, batch_size)

Dataset Rows: 27728
Starting to process 1000 rows in batches of 50.
Processing batch starting at row 1...
  Processed QA pair 1 successfully.
  Processed QA pair 2 successfully.
  Processed QA pair 3 successfully.
  Processed QA pair 4 successfully.
  Processed QA pair 5 successfully.
  Processed QA pair 6 successfully.
  Processed QA pair 7 successfully.
  Processed QA pair 8 successfully.
  Processed QA pair 9 successfully.
  Processed QA pair 10 successfully.
  Processed QA pair 11 successfully.
  Processed QA pair 12 successfully.
  Processed QA pair 13 successfully.
  Processed QA pair 14 successfully.
  Processed QA pair 15 successfully.
  Processed QA pair 16 successfully.
  Processed QA pair 17 successfully.
  Processed QA pair 18 successfully.
  Processed QA pair 19 successfully.
  Processed QA pair 20 successfully.
  Processed QA pair 21 successfully.
  Processed QA pair 22 successfully.
  Processed QA pair 23 successfully.
  Processed QA pair 24 successfully.
  Processed QA 

In [12]:
new_df = pd.read_json('/kaggle/working/generated_questions.json')
new_df.head()

# Convert list-type columns to comma-separated strings
new_df['correct_options'] = new_df['correct_options'].apply(lambda x: '; '.join(x))
new_df['distractor_options'] = new_df['distractor_options'].apply(lambda x: '; '.join(x))

# Save as CSV (optional)
new_df.to_csv('my_dataset.csv', index=False)


In [13]:
for index, rows in new_df.iloc[:20].iterrows():
    question = rows['question']
    answer = rows['correct_options']
    distractors = rows['distractor_options']
    
    print(f"--- QA Pair {index + 1} ---")
    print(f"Q: {question}")
    print(f"A: {answer}")
    print(f"WA: {distractors}\n")

--- QA Pair 1 ---
Q: Which of the following are symptoms of gastritis? (Select all that apply)
A: Nausea or recurrent upset stomach is a common symptom of gastritis.; Abdominal bloating can occur in individuals with gastritis.; A burning or gnawing sensation in the stomach may happen between meals or at night.; Vomiting blood or material resembling coffee grounds is a potential symptom.; Black, tarry stools can indicate gastritis.
WA: Gastritis is characterized by a persistent dry cough.; Gastritis typically leads to increased energy and appetite.; It is primarily associated with respiratory symptoms.

--- QA Pair 2 ---
Q: Which of the following treatments for gastritis are correct? (Select all that apply)
A: Taking antacids and medications like proton pump inhibitors can help reduce stomach acid.; A regimen of antibiotics along with an acid-blocking drug is prescribed for H. pylori infection-related gastritis.; B12 vitamin shots are administered if gastritis is due to pernicious anemi